In [8]:
import tiktoken
import os
import pandas as pd
import openai
import time
from ast import literal_eval

In [21]:
openai.organization = os.environ['OPENAI_ORG_KEY']
openai.api_key = os.environ['OPENAI_API_KEY']


# Start from begginning
#df = pd.read_excel("liste_fichiers_new.xlsx")

source = 'vLex'
# Read the Excel source
#df = pd.read_excel(f"src/{source}/liste_fichiers{source}.xlsx")

# Start from last processed
df = pd.read_csv(f"eval_sentiment_{source}.csv")
df['feedback_abstract'] = df['feedback_abstract'].apply(literal_eval)

In [25]:
def num_tokens_from_string(string: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")
    num_tokens = len(encoding.encode(string))
    return num_tokens

In [10]:
def content_eval(part, title, content):
    # 'part' can be 'abstract', 'results' or 'conclusion'
    
        
#     role = f"""Given the overarching research question - 'What is the current state of research on the challenges, limitations, and potential impact of using causal AI techniques for automatic legal language processing and its application in legal decision-making compared to non-causal AI models relying on correlations?' - and the more specific sub-questions, I evaluate the relevance of the following article based on its {part} and title. I consider if the article addresses the following points:

#     1. The challenges in interpreting and understanding the meaning of legal language, and how these impact the ability of AI models to analyze and make decisions based on legal texts.
#     2. The relationship between causal inference algorithms and the process of legal language understanding.
#     3. Methods to overcome the lack of empirical context in the causal analysis of legal texts, and suggestions for collecting data on a sufficient set of variables to block all the backdoor paths between the intervention and the outcome.
#     4. The challenges and limitations in conducting automatic analysis of legal corpora and whether it proposes a theoretical framework to evaluate the causal links in the models produced.
#     5. The potential impact of using causal AI techniques in the field of automatic legal language processing and its application in legal decision-making compared to traditional AI models relying on correlations.

# I provide a brief reasoning for my evaluation."""

    role = """
In light of our primary research query - 'What is the prevailing status of studies concerning the difficulties, constraints, and potential effects of applying causal AI methods to automatic legal language processing and their role in legal decision-making, in comparison to non-causal AI models that rely on correlations?' - and the associated, more detailed, sub-questions, I express my feelings about the relevance of the selected article based on its {part} and title. 

Rather than simply stating the relevance as "positive" or "negative", I describe how the article makes me feel in a more human way, as they would on social media like Twitter or Facebook. I express feelings of excitement, disappointment, surprise, interest, confusion, optimism, skepticism, and other emotions that arise while considering the relevance of the article. I try to weave these emotions throughout my response, rather than just stating them at the beginning.

The evaluation hinges on whether the article addresses these specific aspects:
     1. The challenges in interpreting and understanding the meaning of legal language, and how these impact the ability of AI models to analyze and make decisions based on legal texts.
     2. The relationship between causal inference algorithms and the process of legal language understanding.
     3. Methods to overcome the lack of empirical context in the causal analysis of legal texts, and suggestions for collecting data on a sufficient set of variables to block all the backdoor paths between the intervention and the outcome.
     4. The challenges and limitations in conducting automatic analysis of legal corpora and whether it proposes a theoretical framework to evaluate the causal links in the models produced.
     5. The potential impact of using causal AI techniques in the field of automatic legal language processing and its application in legal decision-making compared to traditional AI models relying on correlations.

Both the polarity (positive/negative) and the intensity (strength) of emotion will be expressed in my language, with punctuation such as exclamation points to increase the intensity of sentiment, capitalization of words, emotional degree modifiers and and shifters.
""" 
    prompt = f"Title: " + title + f"""

{part.capitalize()}: """ + content 
    
    answer = f"Based on the title and {part}, "

    while True:
        try:
            completions = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=[
                { "role": "assistant", "content": role
                },
                { "role": "user", "content": prompt},
                { "role": "assistant", "content": answer}
                ],
                #max_tokens=num_tokens_from_string(role + " " + prompt + " " + answer) + 200,
                max_tokens = 200,
            )
            return completions['choices'][0]['message']['content']
        except openai.error.RateLimitError:  # handle the specific RateLimitError
            print('API overload, waiting for 10 seconds...')
            time.sleep(10)
        except Exception as e:
            # Handle any other exceptions that might occur
            print("Error:", str(e))
            time.sleep(10)

In [41]:
parts = ['abstract']

# Creating new columns with empty lists
for part in parts:
    df['feedback_' + part] = [[] for _ in range(len(df))]

In [22]:
for i in df['feedback_abstract']:
    print(len(i))

50
50
50
50
50
0
0
0


In [23]:
df

,Unnamed: 0,name,abstract,feedback_abstract
0,0,"Agentes artificiales, opacidad tecnológica y ...",ABSTRACT: The paper focuses on the criminal li...,[I feel that this article is highly relevant t...
1,1,Inteligencia artificial y ADR- acceso a la jus...,Abstract: Artificial intelligence and Alternat...,[I feel intrigued and interested about the pot...
2,2,Inteligencia artificial y Derecho civil- líne...,"ABSTRACT: Let's be honest, robots scare us. Th...",[I feel that the article is relevant to the to...
3,3,La decisión judicial automatizada_ entre la c...,"ABSTRACT: Recent technological advances, and e...",[I feel interested in this article's analysis ...
4,4,"La decisión robótica_ algoritmos, interpreta...",AbSTRAcT. Predictability and predictive justic...,[I feel interested and intrigued by the articl...
5,5,La responsabilidad civil derivada de los daño...,NaN,[]
6,6,"Resolución de 16 de diciembre de 2020, ... Or...",NaN,[]
7,7,Resoluciones. RESOLUCIÓN Nro. 007-2022-CNP. I...,NaN,[]


In [20]:
for part in parts:
    # Apply function if rows is not empty string nor NaN
    for idx, row in df[df[part].notna() & (df[part] != '')].iterrows():
        title = row['name']
        content = row[part]
        
        feedback_col_name = 'feedback_' + part
        # start from the next index if the list is not empty
        start_index = len(df.at[idx, feedback_col_name]) if df.at[idx, feedback_col_name] else 0

        for i in range(start_index, 50):
            feedback = content_eval(part, title, content)
            # append feedback to the list
            df.at[idx, feedback_col_name] = df.at[idx, feedback_col_name] + [feedback] if df.at[idx, feedback_col_name] else [feedback]
        
            # Save progress after each successful iteration
            df.to_csv(f'eval_sentiment_{source}.csv')

API overload, waiting for 10 seconds...
API overload, waiting for 10 seconds...
API overload, waiting for 10 seconds...
API overload, waiting for 10 seconds...
API overload, waiting for 10 seconds...
API overload, waiting for 10 seconds...
API overload, waiting for 10 seconds...
API overload, waiting for 10 seconds...
API overload, waiting for 10 seconds...


In [168]:
tokens = 0
for part in parts:
    # Appliquer la fonction à chaque ligne non vide et non NaN
    for idx, row in df[df[part].notna() & (df[part] != '')].iterrows():
        title = row['name']
        content = row[part]
        
        data = []
        tokens_p = num_tokens_from_string(part + ' ' + title + ' ' + content) + 275 + 200
        for i in range(0,49):
            tokens+= tokens_p

tokens

5101880

In [169]:
0.002*tokens/1000


10.20376